# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [65]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [66]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [33]:
wine_df.info()
178 # Your answer here

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  class

178

_(ii)_ How many variables (columns) does the dataset contain?

In [4]:
14 # Your answer here

14

_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [11]:
wine_df['class'].unique() 
print('The class column in wine data has variable type as int32 and it has 3 unique values 0, 1 and 2') # Your answer here

class column has variable type as int32 and it has 3 unique values 0, 1 and 2



_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [34]:
print('There are 13 other variables in the wine dataset')# Your answer here

There are 13 other variables in the wine dataset


You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [67]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

It's important to standardize predictor variables so as to remove issues that occur due to scale of the variables. Without scaling, a variable with values between 1000 and 2000 will have more wieght than a variable between 1 and 2. 

(ii) Why did we elect not to standard our response variable `Class`?

We did not standardize our response variable because it is being competing with any other variable for model wieght. It's just one variable.

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

In [36]:
np.random.seed(42)

Seed is helpful when replicating results of a previously built model. If one uses seed, although the variable is randomly generated, it will generate the same output and the results can be compared with an example, A great use case would be when one is submitting the model for peer review. 

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [73]:
# set a seed for reproducibility
np.random.seed(123)

# split the data into a training and testing set. hint: use train_test_split !

wine_df_train, wine_df_test = train_test_split(wine_df, train_size=0.75, shuffle = True, stratify=wine_df['class']) 

predictors_train = wine_df_train.copy() #made a copy of winedftrain
predictors_test = wine_df_test.copy() #made a copy of winedftest

response_columns=['class']            #define response and predictor columns
predictor_columns=predictors_train.columns.difference(response_columns)

# Standardize the predictors
scaler = StandardScaler()
predictors_train[predictor_columns] = scaler.fit_transform(predictors_train[predictor_columns]) # standardized the predictor columns in the associated copy
predictors_test[predictor_columns] = scaler.transform(predictors_test[predictor_columns]) # standardized the predictor columns in the associated copy

# Display the head of the standardized training dataset
print(predictors_train.head())

print(predictors_test.head())

     alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
78 -0.836890   -1.269530 -1.395147          -1.340743   2.504594   
0   1.544128   -0.609576  0.319643          -1.102865   1.895120   
15  0.792228   -0.517915  1.284213          -0.627111   0.879332   
13  2.195775   -0.591243  0.176744          -2.351721  -0.542772   
14  1.732103   -0.462919  0.141019          -2.173313   0.202139   

    total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
78      -0.567856   -0.154709             -0.102985         2.022775   
0        0.830752    1.042276             -0.677109         1.213691   
15       0.908453    0.893889             -0.513074        -0.215117   
13       1.296955    1.665499              0.553158         2.108848   
14       1.607757    1.616037             -0.595092         2.367067   

    color_intensity       hue  od280/od315_of_diluted_wines   proline  class  
78        -0.757922  0.493106                     -0.414179 -0.014251      1  


In [69]:
predictors_train

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
78,-0.836890,-1.269530,-1.395147,-1.340743,2.504594,-0.567856,-0.154709,-0.102985,2.022775,-0.757922,0.493106,-0.414179,-0.014251,1
0,1.544128,-0.609576,0.319643,-1.102865,1.895120,0.830752,1.042276,-0.677109,1.213691,0.239076,0.403719,1.873365,0.956541,0
15,0.792228,-0.517915,1.284213,-0.627111,0.879332,0.908453,0.893889,-0.513074,-0.215117,0.977923,1.476361,0.395697,1.711601,0
13,2.195775,-0.591243,0.176744,-2.351721,-0.542772,1.296955,1.665499,0.553158,2.108848,0.132255,1.342281,0.182572,1.218500,0
14,1.732103,-0.462919,0.141019,-2.173313,0.202139,1.607757,1.616037,-0.595092,2.367067,1.066941,1.118814,0.566197,2.442006,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,-1.551195,0.279530,2.070158,0.205460,0.269859,-0.800958,0.023355,1.947460,-0.920914,-0.579887,1.252894,-0.144220,-0.454959,1
101,-0.498534,-0.948719,-1.573771,-0.240560,-0.745930,-1.267160,-0.639438,-0.595092,-0.404477,-1.180757,0.403719,0.239405,-0.593644,1
12,0.942608,-0.591243,0.248193,-0.983927,-0.678211,0.519951,0.745503,-0.595092,0.387392,0.221273,0.895347,0.424114,1.742420,0
61,-0.448408,-0.930387,-1.145074,-0.746049,0.066701,-0.381375,-0.589976,1.373336,-1.661140,0.288036,0.135559,-1.437179,-0.938814,1


In [70]:
predictors_test

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
102,-0.793705,0.194544,0.070077,0.301647,-0.315744,0.319926,0.013835,-0.151778,-0.553456,-0.853950,-0.756952,1.026210,-1.023027,1
84,-1.387007,-1.136546,0.574633,-0.645384,-0.640508,-0.330362,0.119358,-1.072679,1.375192,-0.753522,-0.798901,0.609177,-0.729757,1
96,-1.422605,-0.087033,1.247375,0.459486,2.607140,-1.414176,-1.168022,-1.686613,-0.089839,-0.974463,-0.127712,-0.530713,-0.354229,1
65,-0.758107,-0.863502,0.490541,-0.613817,-0.315744,0.067036,0.583659,0.078447,0.874485,-0.130869,0.879071,-0.475108,-0.164676,1
79,-0.366528,1.406177,-0.182201,0.933001,-0.072170,0.807642,0.478136,0.538898,0.633404,-0.946343,0.879071,0.678682,-0.933615,1
17,0.974333,-0.556327,0.742819,-0.014030,1.064507,1.024404,1.375081,0.308672,0.206876,0.672555,0.627375,-0.099779,1.451884,0
109,-1.659925,-0.744045,1.079189,-0.014030,-0.640508,0.645070,0.868571,-0.535487,1.634817,-0.914207,-0.085763,0.859396,-0.157523,1
113,-1.897246,-1.264535,0.238263,0.301647,-1.127656,0.175417,-0.091688,0.462156,-0.312375,-0.741471,0.501527,-0.461207,-1.037333,1
28,1.021797,-0.274751,1.499653,-0.203436,0.414977,1.024404,0.921333,0.078447,0.281055,-0.171040,1.130767,1.054012,0.682945,0
159,0.559022,-0.471001,0.826911,0.775163,-1.046464,0.392180,-1.051947,1.229574,1.263924,2.741370,-1.721785,-1.197965,-0.372111,2


#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [71]:
knn = KNeighborsClassifier(n_neighbors=1) #intialised a KNN classifier

parameter_grid = {
    "n_neighbors": range(1, 50, 3),
}                     # defined a parameter grid with range 50 and increment of 3           

wine_tune_grid= GridSearchCV(
    estimator=knn, 
    param_grid=parameter_grid, 
    cv=10)                                          #deined a cross validation grid

wine_tune_grid.fit(                                 #create the tune gris
    predictors_train[predictor_columns],
    predictors_train['class']
)

wine_tune_grid.best_params_ # identify the best param

{'n_neighbors': 13}

#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [72]:
# fit the model on the k=13 data
knn2 = KNeighborsClassifier(n_neighbors=13) 
X = predictors_train[predictor_columns]
y = predictors_train["class"]
knn2.fit(X, y)

predictors_test['predicted']=knn2.predict(predictors_test[predictor_columns]) #create an additional column with predicted values in test data

acc = accuracy_score(y_true=predictors_test['class'],y_pred=predictors_test['predicted'], normalize=True) # compute the accuracy score on test data using accuracy score function

acc

0.9555555555555556

# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `11:59 PM - 05/18/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/LCR/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-6-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
